# Аннотация

Скрипт формирует список узлов ***course_activity_nodes.csv*** и ребер ***course_activity_edges.csv*** на основе лог-файла электронного учебного курса для дальнейшей визуализации динамического графа в Gephi.

# Код

## 1 Библиотеки

In [1]:
import pandas as pd

## 2 Исходный лог-файл

In [2]:
df = pd.read_csv('course_activity_log.csv')

In [3]:
df.shape

(1055, 3)

In [4]:
df.head(5)

,datetime,student,task
0,2019-09-10 19:38,Олег,Самостоятельная работа №1. Задачи анализа данн...
1,2019-09-11 15:35,Надежда,Лабораторная работа №1. Python
2,2019-09-11 15:35,Надежда,Лабораторная работа №1. Python
3,2019-09-11 15:36,Степан,Лабораторная работа №1. Python
4,2019-09-11 15:36,Степан,Лабораторная работа №1. Python


## 3 Расчет количества обращенй студентов к элементам учебного курса

### Извлечение даты

In [5]:
df['datetime'] = df['datetime'].str.slice(0,10)
df.rename(columns={'datetime': 'date'}, inplace=True)

In [6]:
df.head(5)

,date,student,task
0,2019-09-10,Олег,Самостоятельная работа №1. Задачи анализа данн...
1,2019-09-11,Надежда,Лабораторная работа №1. Python
2,2019-09-11,Надежда,Лабораторная работа №1. Python
3,2019-09-11,Степан,Лабораторная работа №1. Python
4,2019-09-11,Степан,Лабораторная работа №1. Python


### Группировка и подсчет

In [7]:
df_group = df.groupby(['date', 'student', 'task']).size().reset_index().rename(columns={0: 'number'})

In [8]:
df_group.head()

,date,student,task,number
0,2019-09-10,Олег,Самостоятельная работа №1. Задачи анализа данн...,1
1,2019-09-11,Дмитрий,Лабораторная работа №1. Python,6
2,2019-09-11,Екатерина,Лабораторная работа №1. Python,6
3,2019-09-11,Кирилл,Лабораторная работа №1. Python,6
4,2019-09-11,Надежда,Лабораторная работа №1. Python,6


## Формирование списка вершин

In [9]:
nodes = []

In [10]:
students = df_group['student'].unique().tolist()
students[:5]

['Олег', 'Дмитрий', 'Екатерина', 'Кирилл', 'Надежда']

In [11]:
nodes.extend([(student, 'student') for student in students])
nodes[:5]

[('Олег', 'student'),
 ('Дмитрий', 'student'),
 ('Екатерина', 'student'),
 ('Кирилл', 'student'),
 ('Надежда', 'student')]

In [12]:
tasks = df['task'].unique().tolist()
tasks[:5]

['Самостоятельная работа №1. Задачи анализа данных и типы данных',
 'Лабораторная работа №1. Python',
 'Самостоятельная работа №2. Установка Anaconda',
 'Лабораторная работа №2. Линейная алгебра и библиотека numpy',
 'Лабораторная работа №3 (часть 1)']

In [13]:
nodes.extend([(task, 'task') for task in tasks])
nodes[-5:]

[('Тест 5. Классификация', 'task'),
 ('Лабораторная работа №5 (часть 2)', 'task'),
 ('Тест 4. Регрессия', 'task'),
 ('Лабораторная работа №6. Разработка системы анализа данных', 'task'),
 ('Лабораторная работа №7. Разработка системы анализа данных (индивидуальный проект)',
  'task')]

In [14]:
df_nodes = pd.DataFrame(nodes, columns=['id', 'node_type'])

In [15]:
df_nodes.shape

(23, 2)

In [16]:
df_nodes.sample(5)

,id,node_type
13,Лабораторная работа №2. Линейная алгебра и биб...,task
16,Лабораторная работа №4. Регрессия,task
15,Лабораторная работа №3 (часть 2),task
17,Лабораторная работа №5 (часть 1),task
7,Светлана,student


In [17]:
df_nodes.to_csv('course_activity_nodes.csv', index=False)

## Формирование списка ребер с учетом динамики

### Cписок ребер

In [18]:
df_group.head()

,date,student,task,number
0,2019-09-10,Олег,Самостоятельная работа №1. Задачи анализа данн...,1
1,2019-09-11,Дмитрий,Лабораторная работа №1. Python,6
2,2019-09-11,Екатерина,Лабораторная работа №1. Python,6
3,2019-09-11,Кирилл,Лабораторная работа №1. Python,6
4,2019-09-11,Надежда,Лабораторная работа №1. Python,6


In [19]:
df_edges = df_group[['student', 'task']].drop_duplicates()

In [20]:
df_edges.head()

,student,task
0,Олег,Самостоятельная работа №1. Задачи анализа данн...
1,Дмитрий,Лабораторная работа №1. Python
2,Екатерина,Лабораторная работа №1. Python
3,Кирилл,Лабораторная работа №1. Python
4,Надежда,Лабораторная работа №1. Python


### Добавление признаков динамики

In [21]:
timestamps = []
weights = []

In [22]:
for _, edge in df_edges.iterrows():
    df_selection = df_group[(df_group['student'] == edge['student']) & (df_group['task'] == edge['task'])][['date', 'number']]
    data = df_selection.values.tolist()
    str_timestamps = '<[' + ', '.join([rec[0] for rec in data]) + ']>'
    str_weights = '<' + '; '.join(['[' + rec[0] + ', ' + str(rec[1]) + ']' for rec in data]) + '>'
    timestamps.append(str_timestamps)
    weights.append(str_weights)

In [23]:
df_edges['timeset'] = timestamps
df_edges['weight'] = weights

In [24]:
df_edges.rename(columns={'student':'source', 'task':'target'}, inplace=True)
df_edges.reset_index(drop=True, inplace=True)
df_edges.index.name = 'id'

In [25]:
df_edges.head()

,source,target,timeset,weight
id,,,,
0,Олег,Самостоятельная работа №1. Задачи анализа данн...,"<[2019-09-10, 2019-09-15]>","<[2019-09-10, 1]; [2019-09-15, 7]>"
1,Дмитрий,Лабораторная работа №1. Python,"<[2019-09-11, 2019-09-14]>","<[2019-09-11, 6]; [2019-09-14, 1]>"
2,Екатерина,Лабораторная работа №1. Python,<[2019-09-11]>,"<[2019-09-11, 6]>"
3,Кирилл,Лабораторная работа №1. Python,"<[2019-09-11, 2019-09-16, 2019-10-11, 2019-10-...","<[2019-09-11, 6]; [2019-09-16, 1]; [2019-10-11..."
4,Надежда,Лабораторная работа №1. Python,<[2019-09-11]>,"<[2019-09-11, 6]>"


In [26]:
df_edges.to_csv('course_activity_edges.csv')